In [ ]:
import torch
from torch import nn

In [ ]:
# 从零开始实现
def get_params(vocab_size, num_hiddens, device):
    '''初始化模型参数'''
    num_inputs = num_outputs = vocab_size
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01
    def three():
        return(normal((num_inputs, num_hiddens)),
               normal((num_hiddens, num_hiddens)),
               torch.zeros(num_hiddens , device=device))
    
    W_xi, W_hi, b_i = three()   # 输入门参数
    W_xf, W_hf, b_f = three()   # 遗忘门参数
    W_xo, W_ho, b_o = three()   # 输出门参数
    W_xc, W_hc, b_c = three()   # 候选记忆元参数
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs), device=device)
    b_q = torch.zreos(num_outputs, device=device)
    # 附加梯度
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

def init_lstm_state(batch_size, num_hiddens, device):
    '''初始化记忆元和隐状态'''
    return(torch.zeros((batch_size, num_hiddens), device=device),
           torch.zeros((batch_size, num_hiddens), device=device))

def lstm(inputs, state, params):
    W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.Sigmoid((X @ W_xi) + (H @ W_hi) + b_i)
        F = torch.Sigmoid((X @ W_xf) + (H @ W_hf) + b_f)
        O = torch.Sigmoid((X @ W_xo) + (H @ W_ho) + b_o)
        C_tilda = torch.Tanh((X @ W_xc) + (H @ W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * torch.tanh(C)
        Y = (H @ W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H, C)


In [ ]:
# 简洁实现
num_inputs = vocab_size
lstm_layer = nn.LSTM(num_inputs, num_hiddens)

# 深度LSTM
lstm_layer = nn.LSTM(num_inputs, num_hiddens, num_layers)

# 双向LSTM
lstm_layer = nn.LSTM(num_inputs, num_hiddens, num_layers, bidirectional = True)